In [1]:
# !pip install --upgrade bokeh

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, output_file, show
import pandas as pd
from bokeh.models.sources import ColumnDataSource

output_notebook()

Loading BokehJS ...

#### Download the Automobile dataset
<b>Dataset location:</b> https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

This contains data about imported automobiles available in the United States in the year 1985

In [3]:
cars = pd.read_csv('datasets/imports-85.data')
cars.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


#### Remove any rows containing missing data

In [4]:
cars = cars.dropna(how = 'any')

#### Convert the price and horsepower fields to numeric
They are initially considered strings (possibly due to missing data), so we convert them to numbers

In [5]:
cars['price'] = pd.to_numeric(cars['price'], 
                              errors='coerce')

cars['horsepower'] = pd.to_numeric(cars['horsepower'], 
                                   errors='coerce')

#### Create a ColumnDataSource from the data

In [6]:
data_source = ColumnDataSource(cars)

#### Define tooltips for the plot
We will creat a scatter plot with markers representing each car. When hovering over the markers, we will display the make of the car along with its price and horsepower.

In [7]:
tooltips = [('Make', '@make'),
            ('Price', '@price'),
            ('Horsepower', '@horsepower'),
           ]

#### Define the figure
This includes the tooltips we have created

In [8]:
p = figure(plot_width = 600, 
           plot_height = 300,
           
           tooltips = tooltips
          )

#### Use circles to plot the cars' horsepower vs their price
Hover over the markers to view the details listed in the tooltip

In [9]:
p.circle(x = 'price', 
         y = 'horsepower', 
         
         size = 10,
         
         source = data_source
        )

show(p)

#### Formatting the tooltip data
We perform two changes here:
* We add a field for the body style of the car. Since the field name 'body-style' contains a '-' sign, we need to enclose it in curly braces
* We format the price field to include a '&#36;' sign at the beginning, and for commas to be included for every 3 digits to the left of the decimal. There will also be two digits to the right of the decimal

In [10]:
tooltips = [('Make', '@make'),
            ('Style', '@{body-style}'),
            ('Price', '$@price{0,0.00}'),
            ('Horsepower', '@horsepower'),
           ]

#### Re-draw the plot with the new tooltips
Hover over the markers to view the changes

In [11]:
p = figure(plot_width = 600, 
           plot_height = 300,
           
           tooltips = tooltips
          )

p.circle(x = 'price', 
         y = 'horsepower', 
         
         size = 10,
         
         source = data_source
        )

show(p)

#### The toolbar location can be changed
By default, it appears to the right of the plot. We move it to the bottom

In [12]:
p.toolbar_location = 'below'

show(p)

#### We can set which tools to include in the toolbar
The full list of tools is available here: https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html

In [13]:
toolbox = ['pan',
           'box_zoom',
           'box_select',
           'save',
           'reset',
           'hover'
          ]

#### Add the new toolbox to the figure
* specify the <b>tools</b> property to include our toolbox
* setting <b>toolbar_sticky</b> to False will ensure that the toolbar lies outside the plot dimensions. Previously, the toolbar used up part of the 300 screen size units of the plot_height. Now the toolbar will not be included within those dimensions, freeing up more space for the graph

In [14]:
p = figure(plot_width = 600, 
           plot_height = 300,
           
           tooltips = tooltips,
           toolbar_location = 'below',
           
           toolbar_sticky = False,
           
           tools = toolbox
          )

#### Re-draw the plot
Notice the tools within the toolbar and explore the Box Select tool. The graph should also be a tad larger

In [15]:
p.circle(x = 'price', 
         y = 'horsepower', 
         
         size = 10,
         
         source = data_source
        )

show(p)

## Edit Tools
There are a number of tools available to edit the plot. These effectively modify the underlying data source to reflect the changes made to the plot. 

The available Edit tools include:
* BoxEditTool to edit Rect glyphs
* PolyDrawTool and PolyEditTool to draw and edit Patches and Multiline glyphs
* PointDrawTool to draw and edit XYGlyph types which can serve as markers (circle, square etc.) 

We explore the PointDrawTool here

In [16]:
from bokeh.models.tools import PointDrawTool

#### Define a figure
The same as the one used previously

In [17]:
p = figure(plot_width = 600, 
           plot_height = 300,
           
           tooltips = tooltips,
           toolbar_location = 'below',
           
           toolbar_sticky = False,
           
           tools = toolbox
          )

#### Define the circles as before, but store them in a variable

In [18]:
circles = p.circle(x = 'price', 
                   y = 'horsepower', 
                   
                   size = 10,
                   source = data_source
        )

#### Create a PointDrawTool for the circles we just defined
We add the circle to a list of renderers for the PointDrawTool. This list can contain multiple XGlyph types

In [19]:
point_tool = PointDrawTool(renderers=[circles])

#### Add the PointDrawTool to the figure
Upon selecting the PointDrawTool from the toolbar, some of the operations which can be performed are:
* drag an existing point to change its location on the graph
* add a new point by clicking at some spot on the graph - only the values represented in the X and Y axes are populated though
* select a point and press BACKSPACE on your keboard (DELETE in a Mac) to delete the point from the graph

Once these changes are made to the plot, it can be saved and downloaded as a 

In [20]:
p.add_tools(point_tool)

show(p)